In [ ]:
# Import necessary libraries
import torch
from torch import nn
from sklearn.datasets import make_moons
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import random

# Set random seeds for reproducibility
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Step 1: Create a binary classification dataset
# Create a dataset with Scikit-Learn's make_moons()
X, y = make_moons(n_samples=1000, noise=0.1, random_state=RANDOM_SEED)

# Turn data into a DataFrame
df = pd.DataFrame({"X1": X[:, 0], "X2": X[:, 1], "label": y})

# Visualize the data on a scatter plot
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu)
plt.xlabel("X1")
plt.ylabel("X2")
plt.title("make_moons data")
plt.show()

# Turn data into tensors of dtype float
X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Step 2: Build a Model
class MoonModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(2, 16)
        self.layer_2 = nn.Linear(16, 16)
        self.layer_3 = nn.Linear(16, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer_1(x))
        x = self.relu(self.layer_2(x))
        return self.layer_3(x)

# Instantiate the model
model = MoonModelV0().to(device)

# Step 3: Setup Loss Function and Optimizer
loss_fn = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss with logits
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 4: Training and Testing Loop
epochs = 1000

# Lists to store metrics for visualization
train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []

for epoch in range(epochs):
    # Training
    model.train()
    y_logits = model(X_train).squeeze()
    y_pred = torch.sigmoid(y_logits)
    
    # Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train)
    acc = torch.eq(y_pred.round(), y_train).float().mean().item()
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Testing
    model.eval()
    with torch.inference_mode():
        test_logits = model(X_test).squeeze()
        test_pred = torch.sigmoid(test_logits)
        test_loss = loss_fn(test_logits, y_test)
        test_acc = torch.eq(test_pred.round(), y_test).float().mean().item()
    
    # Store metrics
    train_losses.append(loss.item())
    test_losses.append(test_loss.item())
    train_accuracies.append(acc)
    test_accuracies.append(test_acc)
    
    # Print metrics every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Train Loss: {loss:.4f} | Train Acc: {acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

# Step 5: Plot Decision Boundaries
def plot_decision_boundary(model, X, y):
    model.to("cpu")
    X, y = X.to("cpu"), y.to("cpu")
    
    # Create a mesh grid
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101), np.linspace(y_min, y_max, 101))
    
    # Predict on the mesh grid
    X_to_pred_on = torch.from_numpy(np.column_stack((xx.ravel(), yy.ravel()))).float()
    
    model.eval()
    with torch.inference_mode():
        y_logits = model(X_to_pred_on)
    
    y_pred = torch.round(torch.sigmoid(y_logits)).reshape(xx.shape).detach().numpy()
    
    # Plot decision boundary
    plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.show()

# Plot decision boundaries for training and test sets
plot_decision_boundary(model, X_train, y_train)
plot_decision_boundary(model, X_test, y_test)

# Plot training and testing loss/accuracy curves
plt.figure(figsize=(12, 6))

# Loss curves
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.title("Loss Curves")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

# Accuracy curves
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(test_accuracies, label="Test Accuracy")
plt.title("Accuracy Curves")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.show()